## Modules and methods which need to be imported:

In [10]:
import omero
from omero.gateway import BlitzGateway
from omero.model.enums import UnitsTime
from omero.model import PlaneInfoI
from omero.model import TimeI
from getpass import getpass

## Definition to connect to the server

Connect to the server. It is also important to close the connection again to clear up potential resources held on the server. This is done in the `disconnect method.

In [2]:
def connect(hostname, username, password, port):
    """
    Connect to an OMERO server
    :param hostname: Host name
    :param username: User
    :param password: Password
    :return: Connected BlitzGateway
    """
    conn = BlitzGateway(username, password,
                        host=hostname, port = port, secure=True)
    conn.connect()
    conn.c.enableKeepAlive(60)
    return conn


def disconnect(conn):
    """
    Disconnect from an OMERO server
    :param conn: The BlitzGateway
    """
    conn.close()

## Function to return an object that contains the "exposure time" information from an input image:

### Only returns a list for the first z plane and first channel!

Need to modify to handle multiple z/c 

In [11]:
def makeInfoList(conn, image):
    query_service = conn.getQueryService()
    params = omero.sys.ParametersI()
    params.addLong('pid', image.getPixelsId())
    query = "from PlaneInfo as Info where \
            Info.theZ=0 and Info.theC=0 and pixels.id=:pid"
    info_list = query_service.findAllByQuery(query, params, conn.SERVICE_OPTS)
    return(info_list)

## Pass the list to this function to actually change the exposure time:

In [12]:
def changeExpTime(conn, info_list):
    for info in info_list: #info_list has one info object per timepoint
        exp_time = info.exposureTime.getValue()
        exp_time = exp_time/1000
        info.exposureTime = TimeI(exp_time, UnitsTime.MILLISECOND)
    conn.getUpdateService().saveArray(info_list)

In [13]:
def makeImageIdList(conn, datasetId):
    """
    Make a list of Ids from images in the specified dataset
    :param conn: The BlitzGateway
    :param datasetId: The dataset's id
    :return: The Images or None
    """
    dataset = conn.getObject("Dataset", datasetId)
    images = []
    for image in dataset.listChildren():
        images.append(image.id)
    if len(images) == 0:
        return None
    else:
        return images

## Main function that calls all others:

In order the use the methods implemented above in a proper standalone script: Wrap it all up in an doIt method and call it from main:

In [ ]:
def doIt(conn, datasetId):
    image_list = makeImageIdList(conn,datasetId)
    for imageId in image_list:
        loaded_image = conn.getObject("Image", imageId) #Could hange to for loop with known image ids, e.g.
        infolist = makeInfoList(conn, loaded_image)
        changeExpTime(conn, infolist)

def main():
    try:
        hostname = input("Host [omerocloud.med.ualberta.ca]: ") or 'omerocloud.med.ualberta.ca'
        username = input("Username [steve]: ") or 'steve'
        password = getpass("Password:")
        datasetId = int(input("DatasetID []:")) 
        port = 4064
        conn = connect(hostname, username, password, port)
        doIt(conn, datasetId)
    finally:
        if conn:
            disconnect(conn)


if __name__ =="__main__":
    main()